In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

import 

import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report

In [39]:
# Load train and test data
train_data = pd.read_csv("../data/train_data.csv")
test_data = pd.read_csv("../data/test_data.csv")

print(train_data.head())

# Split train and test data into features and targets
train_features = train_data["lyrics"]
train_targets = train_data["most_common_genre"]

test_features = test_data["lyrics"]
test_targets = test_data["most_common_genre"]

                       id                                             lyrics  \
0  1FAmKoufyAXMfzPPs9bsjA  i tied my bandana took my pack from the floor ...   
1  3QvPEv8XjHa73iYhaienWw  i want to live on the moon never see a human a...   
2  5VPFATm85G3P04Q5g8yxqr  bitch you know you can t parallel park anyway ...   
3  7J2jCftItt7htcOUdcMnpt  graceless falling slipping in the cold with no...   
4  4cBPzVIbDIQx0LIyauFAy0  madame morse estate stood five hundred years p...   

       artist_name most_common_genre  \
0  Waylon Jennings           country   
1   Phantom Planet              rock   
2    Isaiah Rashad           hip-hop   
3     Matt Pond PA             indie   
4       Ariel Pink               pop   

                                          genre_list  
0  ['country', 'country', 'rock', 'outlaw', 'coun...  
1                                    ['pop', 'rock']  
2  ['hip-hop', 'rap', 'tennessee', 'hip-hop', 'un...  
3                                ['philly', 'indie']  
4  

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectors_train = vectorizer.fit_transform(train_features)
vectors_test = vectorizer.transform(test_features)
print(vectors_train.shape, vectors_test.shape)

(19052, 57574) (4763, 57574)


In [41]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
clf = MultinomialNB(alpha=.01)
clf.fit(vectors_train, train_targets)
pred = clf.predict(vectors_test)
print(metrics.f1_score(test_targets, pred, average='macro'),
     metrics.accuracy_score(test_targets, pred))

0.3652600973786819 0.431660718034852


In [44]:
import numpy as np
def show_top10(classifier, vectorizer, categories):
     feature_names = np.asarray(vectorizer.get_feature_names())
     for i, category in enumerate(categories):
         top10 = np.argsort(classifier.feature_log_prob_[i])[-10:]
         print("%s: %s" % (category, " ".join(feature_names[top10])))
        
show_top10(clf,vectorizer, train_targets.unique())

country: we your it of me my and to you the
rock: she in my that me it to and you the
hip-hop: like me that in my it to and you the
indie: your we in me my it to and the you
pop: we my in it me and of to you the
rap: that oh we my to it me and the you
christian: me my of in we it and to you the
metal: in that me nigga to and my it you the
punk: your of in my me it to and the you
soul: my it oh and baby to love the me you


In [46]:
from sklearn.svm import LinearSVC
clf = LinearSVC()

clf.fit(vectors_train, train_targets)
pred = clf.predict(vectors_test)
print(metrics.f1_score(test_targets, pred, average='macro'), 
metrics.accuracy_score(test_targets, pred))

0.40328385016391743 0.4400587864791098


In [48]:
vectorizer = TfidfVectorizer(ngram_range=(1,5))
vectors_train = vectorizer.fit_transform(train_features)
vectors_test = vectorizer.transform(test_features)
print(vectors_train.shape, vectors_test.shape)

clf = LinearSVC()

clf.fit(vectors_train, train_targets)
pred = clf.predict(vectors_test)
print(metrics.f1_score(test_targets, pred, average='macro'), 
metrics.accuracy_score(test_targets, pred))

(19052, 10326874) (4763, 10326874)


In [50]:
vectorizer = TfidfVectorizer(ngram_range=(1,5), stop_words='english')
vectors_train = vectorizer.fit_transform(train_features)
vectors_test = vectorizer.transform(test_features)
print(vectors_train.shape, vectors_test.shape)

clf = LinearSVC()

clf.fit(vectors_train, train_targets)
pred = clf.predict(vectors_test)
print(metrics.f1_score(test_targets, pred, average='macro'), 
metrics.accuracy_score(test_targets, pred))

(19052, 6320460) (4763, 6320460)
0.3990220103637426 0.44866680663447406


In [51]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.pipeline import make_pipeline

context_features = FeatureUnion(
        transformer_list=[
            ('word1', TfidfVectorizer(ngram_range=(1, 7),lowercase=True)),
            ('word2', TfidfVectorizer(ngram_range=(1, 3),lowercase=True)),
            ('word3', TfidfVectorizer(ngram_range=(1, 5),lowercase=False)),
            ('char', TfidfVectorizer(lowercase=False, analyzer='char', ngram_range=(2, 3)))])

clf = make_pipeline(context_features, LinearSVC())
clf.fit(train_features, train_targets)
pred = clf.predict(test_features)

print(metrics.f1_score(test_targets, pred, average='macro'), 
metrics.accuracy_score(test_targets, pred))

0.4136638606358827 0.46609279865630904


In [53]:
clf.score(test_features, test_targets)

0.46609279865630904